# 对Research Project 实现聚类

### 环境准备

##### 常用环境导入

In [1]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE

##### 导入matplotlib

In [2]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### 连接数据库

In [3]:
import sqlite3
conn = sqlite3.connect("../db.sqlite3")
cur = conn.cursor()
table_name = cur.execute("select name from sqlite_master where type='table' order by name").fetchall()
table_name

[('Training',),
 ('UserTemp',),
 ('account_confirmstring',),
 ('account_profile_awardhistory',),
 ('account_profile_awardhistory_User',),
 ('account_profile_competitionrecord',),
 ('account_profile_honortitle',),
 ('account_profile_paperrecord',),
 ('account_profile_patentrecord',),
 ('account_profile_publishrecord',),
 ('account_profile_researchproject',),
 ('account_user',),
 ('account_user_groups',),
 ('account_user_user_permissions',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_permission',),
 ('award_info_awardclassification',),
 ('award_info_awardinfo',),
 ('award_info_awardinfo_Major',),
 ('award_info_awardinfopost',),
 ('award_info_category',),
 ('award_info_post',),
 ('award_info_post_tags',),
 ('award_info_post_users_focus',),
 ('award_info_tag',),
 ('captcha_captchastore',),
 ('django_admin_log',),
 ('django_content_type',),
 ('django_migrations',),
 ('django_session',),
 ('grade_info_deptinfo',),
 ('grade_info_gradeinfo',),
 ('grade_info_lessoninfo',),
 ('grade

In [4]:
cur.execute("PRAGMA table_info(account_profile_researchproject)")
cur.fetchall()

[(0, 'id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(256)', 1, None, 0),
 (2, 'projectType', 'varchar(2)', 1, None, 0),
 (3, 'projectWork', 'varchar(256)', 1, None, 0),
 (4, 'SchoolYearInfo_id', 'integer', 1, None, 0),
 (5, 'User_id', 'integer', 1, None, 0)]

In [5]:
csv_head = ["id", "User_id", "projectType"]
record = conn.execute("select {} from account_profile_researchproject".format(",".join(csv_head)))

In [6]:
import csv

In [7]:
def save_csv(record, csv_head, csv_name):
    with open(csv_name, "w", newline="") as f:
        ff = csv.writer(f)
        ff.writerow(csv_head)
        ff.writerows(record)

In [8]:
def read_record(csv_name, csv_head):
    df_record = pd.read_csv(csv_name, header=0, encoding="utf-8")
    mapping = list(map(lambda x:False if x <= 1 else True, [i for i in range(0, len(csv_head))]))
#     df_one_hot = pd.factorize(df_record.loc[:, mapping].copy())
    df_fact = df_record.loc[:, mapping].copy()
    df_one_hot = pd.get_dummies(df_record.loc[:, mapping].copy())
    return df_record, df_one_hot, df_fact

##### save_csv

In [9]:
save_csv(record, csv_head, "research_project.csv")

##### read_csv

In [10]:
df_record, df_record_one_hot, df_record_fact = read_record("research_project.csv", csv_head)
df_record_fact.columns

Index(['projectType'], dtype='object')

### Factorize 或Mapping 给 record赋值

##### Mapping

In [11]:
projectType_mapping = {
    "ng": 1,
    "nm": 2,
    "ns": 3,
    "no": 4,
    "p9": 5,
    "p8": 6,
    "ot": 7
}

df_record_fact["projectType"] = df_record_fact["projectType"].map(projectType_mapping)

##### Factorize

In [ ]:
for i in df_record_fact.columns:
    df_record_fact[i] = pd.Series(pd.factorize(df_record_fact[i])[0])

### 保存CSV

##### one-hot

In [14]:
df_record_one_hot['user_id'] = df_record.loc[:, ["User_id"]]
df_record_one_hot = df_record_one_hot.sort_values("user_id").reset_index(drop=True)
df_record_one_hot.to_csv("research_project_one_hot.csv")

##### factorize

In [16]:
df_record_fact["user_id"] = df_record.loc[:, ['User_id']]
df_record_fact = df_record_fact.sort_values("user_id").reset_index(drop=True)
df_record_fact.to_csv("research_project_fact.csv")